# Analyzing NASA Planetary Exploration Budgets

First we will check all our tables

In [2]:
SELECT *
FROM inflation

,fiscal_year,inflation_adjustment
0,1959,13.937
1,1960,13.363
2,1961,12.948
3,1962,12.450
4,1963,12.029
...,...,...
73,2031,1.000
74,2032,1.000
75,2033,1.000
76,2034,1.000


In [3]:
SELECT *
FROM mission_budgets

,mission,fiscal_year,cost_type,cost_group,cost_MUSD
0,Cassini,1990,Spacecraft,Development/Implementation,29.5
1,Cassini,1991,Launch Support,Launch Vehicle(s) & Support,2.4
2,Cassini,1991,Spacecraft,Development/Implementation,143.0
3,Cassini,1992,Launch Support,Launch Vehicle(s) & Support,20.1
4,Cassini,1992,Spacecraft,Development/Implementation,210.7
...,...,...,...,...,...
1008,Voyager,2019,Operations,Operations & Data Analysis,5.8
1009,Voyager,2020,Operations,Operations & Data Analysis,6.7
1010,Voyager,2021,Operations,Operations & Data Analysis,6.5
1011,Voyager,2022,Operations,Operations & Data Analysis,5.6


In [4]:
SELECT *
FROM mission_details

,mission,mission_full_name,destination,program
0,Cassini,Cassini-Huygens,Outer Planets,
1,CONTOUR,COmet Nucleus TOUR,Small Bodies,Discovery
2,DART,Double Asteroid Redirection Test,Small Bodies,Solar System Exploration
3,DAVINCI,Deep Atmosphere Venus Investigation of Noble g...,Venus,Discovery
4,Dawn,Dawn,Small Bodies,Discovery
5,Deep Impact,Deep Impact,Small Bodies,Discovery
6,Deep Space 1,Deep Space 1,Small Bodies,New Millenium
7,EscaPADE,Escape and Plasma Acceleration and Dynamics Ex...,Mars,SIMPLEx
8,Europa Clipper,Europa Clipper,Outer Planets,Solar System Exploration
9,Galileo,Galileo,Outer Planets,


In [5]:
SELECT MIN(fiscal_year), MAX(fiscal_year)
FROM inflation

,min,max
0,1959,2035


This shows that we have our exploration form 1959 to 2035

### Now, we will check for the total expenditure in all of our exploration

In [7]:
SELECT SUM("cost_MUSD") AS total_cost_MUSD
FROM mission_budgets

,total_cost_musd
0,41406.554506


Since inflation effects the total expenditure, so we have to adjust for it as well

In [9]:
SELECT SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets as m
LEFT JOIN inflation as i
ON m.fiscal_year = i.fiscal_year

,total_cost_musd
0,80992.593983


### Checking for the 3 most expensive mission

In [24]:
SELECT mission, SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets as m
LEFT JOIN inflation as i
ON m.fiscal_year = i.fiscal_year
GROUP BY mission
ORDER BY adjusted_total_cost_MUSD DESC
LIMIT 3

,mission,adjusted_total_cost_musd
0,Viking,7208.541041
1,Surveyor 1 - 7,6333.910560
2,Cassini,5285.431955


### Checking for the years in which we had the most expenditure

In [25]:
SELECT i.fiscal_year, SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets as m
LEFT JOIN inflation as i
ON m.fiscal_year = i.fiscal_year
GROUP BY i.fiscal_year
ORDER BY adjusted_total_cost_MUSD DESC

,fiscal_year,adjusted_total_cost_musd
0,1974,2637.251406
1,1965,2620.062486
2,1966,2496.027447
3,1973,2443.451351
4,1964,2412.935820
...,...,...
72,2033,149.593212
73,2032,149.590728
74,2030,149.516204
75,2034,128.050000


### Checking on which destinations with respect to their expenditure

In [26]:
SELECT md.destination, 
SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets as m
LEFT JOIN inflation as i
ON m.fiscal_year = i.fiscal_year
LEFT JOIN mission_details as md
on m.mission = md.mission
GROUP BY md.destination
ORDER BY adjusted_total_cost_MUSD DESC

,destination,adjusted_total_cost_musd
0,Mars,31725.348594
1,Outer Planets,21827.564786
2,The Moon,12982.887263
3,Small Bodies,7365.860162
4,Venus,4796.703011
5,Mercury,1519.605957
6,Earth-Sun L1,427.220100
7,The Sun,347.404110


### Which group has the most expenditure

In [27]:
SELECT m.cost_group, 
SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets as m
LEFT JOIN inflation as i
ON m.fiscal_year = i.fiscal_year
LEFT JOIN mission_details as md
on m.mission = md.mission
GROUP BY m.cost_group
ORDER BY adjusted_total_cost_MUSD DESC

,cost_group,adjusted_total_cost_musd
0,Development/Implementation,28829.239652
1,Development/Implementation (incl LV),16599.165600
2,Operations & Data Analysis,13421.083087
3,Formulation,8770.964100
4,Launch Vehicle(s) & Support,7930.072962
5,Science & Experiments,3777.250974
6,Project Management/Ground Operations,1664.817608


### How much money was spent on each mission of mars

In [28]:
SELECT m.mission, 
SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
FROM mission_budgets as m
LEFT JOIN inflation as i
ON m.fiscal_year = i.fiscal_year
LEFT JOIN mission_details as md
on m.mission = md.mission
WHERE md.destination = 'Mars'
GROUP BY m.mission
ORDER BY adjusted_total_cost_MUSD DESC

,mission,adjusted_total_cost_musd
0,Viking,7208.541041
1,Mars Sample Return,4309.570000
2,MSL Curiosity,3749.528800
3,Mars Perseverance,2977.926400
4,Mars Observer,1767.490594
5,MER,1587.334067
6,MRO,1426.064600
7,Mariner-Mars 1969 (VI & VII),1321.627043
8,Mariner 8 & 9,1285.142968
9,Mariner-Mars '64 (III & IV),1146.638881
